In [ ]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense, Input, LSTM
from keras.models import Model
from keras.optimizers import Adam
import IPython
import IPython.display
import tensorflow as tf
from tensorflow import keras
from matplotlib import rcParams
from keras.models import Sequential
import scipy
from scipy import io
from scipy.interpolate import interp1d
import os
from sklearn.metrics import accuracy_score, f1_score,mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, LSTM, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dropout, Flatten, Dense, Conv1D, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scipy.io import loadmat
from tqdm import tqdm

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## ok save mean and std, but min max
def interpolate_signal(insignal, len1, len2):

    first_point = insignal[0]
    last_point = insignal[-1]

    x1 = np.linspace(0, len2 - 1, len1)
    f = interp1d(x1, insignal, axis=0, kind='linear')
    x2 = np.linspace(0, len2 - 1, len2)
    interpolated = f(x2)

    interpolated[0] = first_point
    interpolated[-1] = last_point
    
    return interpolated

def list_to_interpolate_preserving_stats(input_list, output_size=145):
    inter_list = []
    for elem in input_list:
        original_mean = np.nanmean(elem)
        original_std = np.nanstd(elem)

        np_int = interpolate_signal(elem, elem.shape[0], output_size)

        central_part = np_int[1:-1]

        interpolated_mean = np.nanmean(central_part)
        interpolated_std = np.nanstd(central_part)

        scaled_central = (central_part - interpolated_mean) * (original_std / interpolated_std) + original_mean

        scaled_int = np.copy(np_int)
        scaled_int[1:-1] = scaled_central

        total_mean = np.nanmean(scaled_int)
        total_std = np.nanstd(scaled_int)
        scaled_int = (scaled_int - total_mean) * (original_std / total_std) + original_mean

        scaled_int[0] = elem[0]
        scaled_int[-1] = elem[-1]
        inter_list.append(scaled_int)
    
    return inter_list

def scaled_data2(input_list):

    global_min = np.nanmin([np.nanmin(traj) for traj in input_list])
    global_max = np.nanmax([np.nanmax(traj) for traj in input_list])

    if global_max == global_min:
        raise ValueError("All trajectories have the same constant value; scaling is not possible.")
    
    normalized_list = []

    for np_array in input_list:
        output_array = 2 * ((np_array - global_min) / (global_max - global_min)) - 1

        #output_array -= output_array[0]

        #range_factor = max(1, np.max(np.abs(output_array)))
        #output_array /= range_factor

        normalized_list.append(output_array)

    return normalized_list, global_max, global_min

In [ ]:
def load_data():
    # Load data
    data = scipy.io.loadmat('../data/trajectories.mat')
    
    # Path
    iid = data['iidind'][2:,]
    rdw = data['rdwind'][2:,]
    posr = np.zeros((iid.shape[0] - 1, iid.shape[1]))
    posw = np.zeros((rdw.shape[0]-1, rdw.shape[1]))
    n_path = posr.shape[1]
    invis = []
    normativ_label_iid = np.zeros((1, n_path))  # ideal responses
    normativ_label_rdw = np.zeros((1, n_path))  # ideal responses
    for tidx in range(n_path):
        # IID
        invisR = int(iid[0, tidx])
        invis.append(invisR)
        r = iid[1:, tidx].copy()
        #r[-invisR + 1:] = np.nan
        r[-invisR + 1:] = 0
        posr[:, tidx] = r
        normativ_label_iid[0, tidx] = 1 if np.nanmean(posr[:, tidx]) < 0 else 0
        # rdw 
        w = rdw[1:, tidx].copy()
        #w[-invisR+1:] = np.nan
        w[-invisR+1:] = 0
        posw[:, tidx] = w
        normativ_label_rdw[0, tidx] = 1 if posw[-invis[tidx], tidx] < 0 else 0
    
    # # Create a list of columns from posr without NaN values
    # path_iid = [np.array(col[~np.isnan(col)]) for col in posr.T]
    
    # # Create a list of columns from posw without NaN values
    # path_rdw = [np.array(col[~np.isnan(col)]) for col in posw.T]
    normativ_label_rdw =normativ_label_rdw.reshape(200,)
    normativ_label_iid =normativ_label_iid.reshape(200,)
    
    ### rdw ###
    normativ_label_rdw_same = np.hstack((normativ_label_rdw[0:50], normativ_label_rdw[150:]))
    normativ_label_rdw_op = normativ_label_rdw[50:150]
    normativ_list_label_rdw_same = [np.array(col[~np.isnan(col)]) for col in normativ_label_rdw_same]
    normativ_list_label_rdw_op = [np.array(col[~np.isnan(col)]) for col in normativ_label_rdw_op]
    normativ_labels_r = np.concatenate((normativ_list_label_rdw_same, normativ_list_label_rdw_op ), axis=0)

    posw_same = np.hstack((posw[:,0:50], posw[:,150:]))
    posw_opposite = posw[:,50:150]
    # Create a list of columns from posr without NaN values
    path_rdw_s = [np.array(col[~np.isnan(col)]) for col in posw_same.T]
    path_rdw_op =[np.array(col[~np.isnan(col)]) for col in posw_opposite.T]

    ### iid ###
    normativ_label_iid_same = np.hstack((normativ_label_iid[0:50], normativ_label_iid[150:]))
    normativ_label_iid_op = normativ_label_iid[50:150]
    normativ_list_label_iid_same = [np.array(col[~np.isnan(col)]) for col in normativ_label_iid_same]
    normativ_list_label_iid_op = [np.array(col[~np.isnan(col)]) for col in normativ_label_iid_op]
    normativ_labels_i = np.concatenate((normativ_list_label_iid_same, normativ_list_label_iid_op ), axis=0)

    posr_same = np.hstack((posr[:,0:50], posr[:,150:]))
    posr_opposite = posr[:,50:150]
    # Create a list of columns from posr without NaN values
    path_iid_s = [np.array(col[~np.isnan(col)]) for col in posr_same.T]
    path_iid_op =[np.array(col[~np.isnan(col)]) for col in posr_opposite.T]    
    
    return path_iid_s, path_iid_op, path_rdw_s, path_rdw_op


def interpolate_signal(insignal, len1, len2):

    first_point = insignal[0]
    last_point = insignal[-1]

    x1 = np.linspace(0, len2 - 1, len1)
    f = interp1d(x1, insignal, axis=0, kind='linear')
    x2 = np.linspace(0, len2 - 1, len2)
    interpolated = f(x2)

    interpolated[0] = first_point
    interpolated[-1] = last_point
    
    return interpolated

def list_to_interpolate_preserving_stats(input_list, output_size=145):
    inter_list = []
    for elem in input_list:
        np_int = interpolate_signal(elem, elem.shape[0], output_size)
        inter_list.append(np_int)
    return inter_list

def scaled_data2(input_list):

    global_min = np.nanmin([np.nanmin(traj) for traj in input_list])
    global_max = np.nanmax([np.nanmax(traj) for traj in input_list])

    if global_max == global_min:
        raise ValueError("All trajectories have the same constant value; scaling is not possible.")
    
    normalized_list = []

    for np_array in input_list:
        output_array = 2 * ((np_array - global_min) / (global_max - global_min)) - 1

        #output_array -= output_array[0]

        #range_factor = max(1, np.max(np.abs(output_array)))
        #output_array /= range_factor

        normalized_list.append(output_array)

    return normalized_list, global_max, global_min

def load_responses(resp_array):
    resp_same =  np.vstack((resp_array[0:50:,], resp_array[150::,]))
    resp_op = resp_array[50:150:,]
    return resp_same,resp_op


def shuffle_data(trajectories, responses,rt):
    indices = np.arange(len(trajectories))  
    np.random.seed(15)
    np.random.shuffle(indices)  
    
    shuffled_trajectories = [trajectories[i] for i in indices]  
    shuffled_responses = responses[indices,: ]  
    shuffled_rt = rt[indices,: ]      
    return shuffled_trajectories, shuffled_responses,shuffled_rt 

def make_dataset (path_iid_s,path_iid_op, resp_iid_s,resp_iid_op,rt_iid_s,rt_iid_op):
    feature_list =[]
    label_list = []
    rt_list=[]
    for i in range(resp_iid_s.shape[1]):
        resp_one_subj_iid_s = resp_iid_s[:, i]
        resp_one_subj_iid_op = resp_iid_op[:, i]
        
        rt_one_subj_iid_s = rt_iid_s[:, i]
        rt_one_subj_iid_op = rt_iid_op[:, i]  
        
        resp_list_iid_s = [np.array(col[~np.isnan(col)]) for col in resp_one_subj_iid_s]
        resp_list_iid_op = [np.array(col[~np.isnan(col)]) for col in resp_one_subj_iid_op]

        rt_list_iid_s = [col[col != 0] for col in rt_one_subj_iid_s]
        rt_list_iid_op = [col[col != 0] for col in rt_one_subj_iid_op]

        scaled_data = np.concatenate((path_iid_s, path_iid_op), axis=0)
        labels = np.concatenate((resp_list_iid_s, resp_list_iid_op), axis=0)
       
        rts = np.concatenate((rt_list_iid_s, rt_list_iid_op), axis=0)

        feature_list.append(scaled_data)
        label_list.append(labels)
        rt_list.append(rts)
    return feature_list, label_list,rt_list

In [ ]:
def shuffle_dataset(feature_list, label_list,rt_list, condition):
    combined_data = list(zip(feature_list, label_list, rt_list)) # Combine features and labels into a single list for shuffling
    np.random.seed(10)
    np.random.shuffle(combined_data) # Shuffle the combined data
    feature_list, label_list, rt_list = zip(*combined_data) # Unpack the shuffled data back into features and labels
    feature_list = list(feature_list) # Convert the shuffled lists back to regular Python lists
    label_list = list(label_list)
    rt_list = list(rt_list)
    
    norm_list= []
    # norm rdw 
    if condition=="rdw":
        norm_list.extend([np.where(feature_list[0][:,-1] > 0, 0, 1)] * 28)    
    elif condition=="iid":
    #norm iid 
        norm_list.extend([np.where(np.mean(feature_list[0].T, axis=0) > 0, 0, 1)] * 28)
    
    X = np.concatenate([f.reshape(f.shape[0], -1) for f in feature_list], axis=0)    
    y = np.concatenate(label_list, axis=0)
    rt = np.concatenate(rt_list, axis=0)
    
    norm = np.array(norm_list)
    all_y = np.concatenate([y[i].reshape(-1) for i in range(len(y))], axis=0)
    all_rt = np.concatenate([rt[i].reshape(-1) for i in range(len(rt))], axis=0)
    all_norm = np.concatenate([norm[i].reshape(-1) for i in range(len(norm))], axis=0)
    # X.shape, y.shape,all_y.shape, all_norm.shape
    return X,all_y,all_rt,all_norm


def remove_nan_indices(X, y, rt):
   
    #X = np.array(X)
    #y = np.array(y)
    #rt = np.array(rt)

    valid_indices = ~np.isnan(rt)
    X_clean = X[valid_indices]
    y_clean = y[valid_indices]
    rt_clean = rt[valid_indices]

    return X_clean, y_clean, rt_clean

def create_recent_points_data(X, y, rt, window_size):
    X_windows = []
    y_windows = []
    rt_windows = []
    
    for i in range(X.shape[0]):  
        trajectory = X[i]
        label = y[i]
        rt_value = rt[i]
        invis = len(np.where(trajectory == 0)[0])-1
        trajectory_non_zero = trajectory[:-invis]
        # last point in window_size 
        window = trajectory_non_zero[-window_size:]
        X_windows.append(window)
        y_windows.append(label)
        rt_windows.append(rt_value)
    
    return np.array(X_windows), np.array(y_windows), np.array(rt_windows)



In [ ]:
path_iid_s, path_iid_op, path_rdw_s, path_rdw_op = load_data()


rt_rdw = pd.read_csv('../data/pros_rt_rdw.csv', header=None).to_numpy()
rt_iid = pd.read_csv('../data/pros_rt_iid.csv', header=None).to_numpy()
resp_rdw = pd.read_csv('../data/resp_rdw.csv', header=None).to_numpy()
resp_iid = pd.read_csv('../data/resp_iid.csv', header=None).to_numpy()

acc_rdw = pd.read_csv('../data/acc_rdw.csv', header=None).to_numpy()
acc_iid = pd.read_csv('../data/acc_iid.csv', header=None).to_numpy()

resp_iid_same,resp_iid_op= load_responses(resp_iid)
resp_rdw_same,resp_rdw_op= load_responses(resp_rdw)

rt_iid_same,rt_iid_op= load_responses(rt_iid)
rt_rdw_same,rt_rdw_op= load_responses(rt_rdw)

norm_acc_iid_same,norm_acc_iid_op = load_responses(acc_iid)
norm_acc_rdw_same,norm_acc_rdw_op = load_responses(acc_rdw)


In [ ]:
######### IID ##########
##interpolation 
path_iid_same_interp_list = list_to_interpolate_preserving_stats(path_iid_s, output_size=150)
path_iid_op_interp_list = list_to_interpolate_preserving_stats(path_iid_op, output_size=150)
# Normalization
scaled_list_iid_same, _, _,  = scaled_data2(path_iid_same_interp_list)  # Transpose for correct shape
scaled_list_iid_op, _, _,  = scaled_data2(path_iid_op_interp_list)  # Transpose for correct shape

######### RDW ##########
##interpolation 
path_rdw_same_interp_list = list_to_interpolate_preserving_stats(path_rdw_s, output_size=150)
path_rdw_op_interp_list = list_to_interpolate_preserving_stats(path_rdw_op, output_size=150)
## Normalization
scaled_list_rdw_same, _, _,  = scaled_data2(path_rdw_same_interp_list)  # Transpose for correct shape
scaled_list_rdw_op, _, _,  = scaled_data2(path_rdw_op_interp_list)  # Transpose for correct shape


scaled_list_iid_same, resp_iid_same,rt_iid_same = shuffle_data(scaled_list_iid_same, resp_iid_same,rt_iid_same)
scaled_list_iid_op, resp_iid_op, rt_iid_op = shuffle_data(scaled_list_iid_op, resp_iid_op,rt_iid_op)

scaled_list_rdw_same, resp_rdw_same,rt_rdw_same = shuffle_data(scaled_list_rdw_same, resp_rdw_same,rt_rdw_same)
scaled_list_rdw_op, resp_rdw_op,rt_rdw_op = shuffle_data(scaled_list_rdw_op, resp_rdw_op,rt_rdw_op)

feature_list_iid, label_list_iid,rt_list_iid = make_dataset(scaled_list_iid_same,scaled_list_iid_op, 
                                                        resp_iid_same,resp_iid_op,rt_iid_same, rt_iid_op)

feature_list_rdw, label_list_rdw,rt_list_rdw = make_dataset (scaled_list_rdw_same,scaled_list_rdw_op,
                                                             resp_rdw_same,resp_rdw_op,rt_rdw_same, rt_rdw_op)


X_iid, all_y_iid, all_rt_iid,all_norm_iid = shuffle_dataset(feature_list_iid, label_list_iid,rt_list_iid,"iid")
X_rdw, all_y_rdw,all_rt_rdw, all_norm_rdw =shuffle_dataset(feature_list_rdw, label_list_rdw,rt_list_rdw,"rdw")

X_iid, all_y_iid, all_rt_iid = remove_nan_indices(X_iid, all_y_iid, all_rt_iid)
X_rdw, all_y_rdw, all_rt_rdw = remove_nan_indices(X_rdw, all_y_rdw, all_rt_rdw)

X_iid_reshaped = X_iid.reshape(X_iid.shape[0], X_iid.shape[1], 1)
X_rdw_reshaped = X_rdw.reshape(X_rdw.shape[0], X_rdw.shape[1], 1)

window_size = 10 


X_iid_recent, y_iid_recent, rt_iid_recent = create_recent_points_data(X_iid_reshaped, all_y_iid, all_rt_iid, window_size)
X_rdw_recent, y_rdw_recent, rt_rdw_recent = create_recent_points_data(X_rdw_reshaped, all_y_rdw, all_rt_rdw, window_size)

print("Shape of X_iid_recent:", X_iid_recent.shape)  #  (path*nsubj, window_size, 1)
print("Shape of y_iid_recent:", y_iid_recent.shape)  # path*nsubj: (5600,)
print("Shape of rt_iid_recent:", rt_iid_recent.shape)  # path*nsubj: (5600,)

np.random.seed(42)
X_combined_resent = np.concatenate((X_iid_recent, X_rdw_recent), axis=0)
y_combined_resent = np.concatenate((y_iid_recent, y_rdw_recent), axis=0)
rt_combined = np.concatenate((rt_iid_recent, rt_rdw_recent), axis=0)

X_combined_full = np.concatenate((X_iid_reshaped, X_rdw_reshaped), axis=0)

X_full_train_comb, X_full_val_comb, X_recent_train_comb, X_recent_val_comb, y_binary_train_comb, y_binary_val_comb, y_time_train_comb, y_time_val_comb = train_test_split(
X_combined_full, X_combined_resent, y_combined_resent, rt_combined, test_size=0.1, random_state=42)


print(f'Training set size: {X_full_train_comb.shape}, {X_recent_train_comb.shape}, {y_binary_train_comb.shape}, {y_time_train_comb.shape}')
print(f'Validation set size: {X_full_val_comb.shape}, {X_recent_val_comb.shape}, {y_binary_val_comb.shape}, {y_time_val_comb.shape}')

In [ ]:

X_full_train_iid, X_full_val_iid, X_recent_train_iid, X_recent_val_iid, y_binary_train_iid, y_binary_val_iid, y_time_train_iid, y_time_val_iid = train_test_split(
X_iid_reshaped, X_iid_recent, y_iid_recent, rt_iid_recent, test_size=0.1, random_state=40)


print(f'Training set size: {X_full_train_iid.shape}, {X_recent_train_iid.shape}, {y_binary_train_iid.shape}, {y_time_train_iid.shape}')
print(f'Validation set size: {X_full_val_iid.shape}, {X_recent_val_iid.shape}, {y_binary_val_iid.shape}, {y_time_val_iid.shape}')

X_full_train_rdw, X_full_val_rdw, X_recent_train_rdw, X_recent_val_rdw, y_binary_train_rdw, y_binary_val_rdw, y_time_train_rdw, y_time_val_rdw = train_test_split(
X_rdw_reshaped, X_rdw_recent, y_rdw_recent, rt_rdw_recent, test_size=0.1, random_state=40)


print(f'Training set size: {X_full_train_rdw.shape}, {X_recent_train_rdw.shape}, {y_binary_train_rdw.shape}, {y_time_train_rdw.shape}')
print(f'Validation set size: {X_full_val_rdw.shape}, {X_recent_val_rdw.shape}, {y_binary_val_rdw.shape}, {y_time_val_rdw.shape}')

In [ ]:
def evaluate_model(model, X_full_trajectory, X_recent_trajectory, y_true_binary, y_true_time):
    predictions = model.predict({'full_trajectory': X_full_trajectory, 'recent_trajectory': X_recent_trajectory})

    binary_predictions = predictions[0]
    time_predictions = predictions[1]

    binary_predictions = (binary_predictions > 0.5).astype(int)

    binary_accuracy = accuracy_score(y_true_binary, binary_predictions)
    precision = precision_score(y_true_binary, binary_predictions)
    recall = recall_score(y_true_binary, binary_predictions)
    f1 = f1_score(y_true_binary, binary_predictions)
    conf_matrix = confusion_matrix(y_true_binary, binary_predictions)

    conf_matrix_percent = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

    rmse_time = np.sqrt(mean_squared_error(y_true_time, time_predictions))
    rt_range = np.max(y_true_time) - np.min(y_true_time)
    relative_rmse_time = rmse_time / rt_range
    #relative_rmse_time = rmse_time / (all_rt_iid.max() - all_rt_iid.min())

    metrics = {
        'Binary Accuracy': binary_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix,
        'Confusion Matrix (%)': conf_matrix_percent,
        'RMSE for RT Predictions': rmse_time,
        'Relative RMSE for RT Predictions': relative_rmse_time
    }

    return metrics, predictions

In [ ]:


def evaluate_model(model, X_full_trajectory, X_recent_trajectory, y_true_binary, y_true_time):

    predictions = model.predict({'full_trajectory': X_full_trajectory, 'recent_trajectory': X_recent_trajectory})

    binary_predictions = predictions[0]  # Binary response predictions
    time_predictions = predictions[1]    # Reaction time predictions

    binary_predictions = (binary_predictions > 0.5).astype(int)

    binary_accuracy = accuracy_score(y_true_binary, binary_predictions)
    precision = precision_score(y_true_binary, binary_predictions)
    recall = recall_score(y_true_binary, binary_predictions)
    f1 = f1_score(y_true_binary, binary_predictions)
    conf_matrix = confusion_matrix(y_true_binary, binary_predictions)


    rmse_time = np.sqrt(mean_squared_error(y_true_time, time_predictions))
    relative_rmse_time = rmse_time / (y_true_time.max() - y_true_time.min())
    metrics = {
        'Binary Accuracy': binary_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix,
        'RMSE for RT Predictions': rmse_time,
        'Relative RMSE for RT Predictions': relative_rmse_time
    }

    return metrics,predictions


def plot_training_history(history):

    rcParams.update({'font.size': 20})  
    rcParams['font.family'] = 'sans-serif'
    rcParams['font.sans-serif'] = ['Calibri']  

    plt.figure(figsize=(27, 9))
    
    plt.subplot(1, 3, 1)
    plt.plot(history.history['response_accuracy'])
    plt.plot(history.history['val_response_accuracy'])
    plt.title('Model Accuracy (Response)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 3, 3)
    plt.plot(history.history['reaction_time_mse'])
    plt.plot(history.history['val_reaction_time_mse'])
    plt.title('Mean Squared Error (RT)')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def compile_and_fit(model, X_full_train, X_recent_train, y_binary_train, y_time_train, 
                    X_full_val, X_recent_val, y_binary_val, y_time_val, patience=40):
 
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
  
    history = model.fit(
        {'full_trajectory': X_full_train, 'recent_trajectory': X_recent_train}, 
        {'response': y_binary_train, 'reaction_time': y_time_train},  
        epochs=100, batch_size=64, 
        validation_data=(
            {'full_trajectory': X_full_val, 'recent_trajectory': X_recent_val},  
            {'response': y_binary_val, 'reaction_time': y_time_val} 
        ),
        callbacks=[early_stopping]
    )
    
    return history

In [ ]:
from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.metrics import accuracy_score, mean_squared_error
import numpy as np
import tensorflow as tf

def create_model():
    full_input = tf.keras.Input(shape=(150, 1), name='full_trajectory')
    recent_input = tf.keras.Input(shape=(10, 1), name='recent_trajectory')
    recent_lstm = tf.keras.layers.LSTM(64, activation='relu', return_sequences=False)(recent_input)
    recent_lstm = tf.keras.layers.Dropout(0.1)(recent_lstm)
    full_flattened = tf.keras.layers.Flatten()(full_input)
    dense_layer = tf.keras.layers.Dense(64, activation='relu')(full_flattened)
    time_output = tf.keras.layers.Dense(1, name='reaction_time', activation='linear')(dense_layer)
    response_output = tf.keras.layers.Dense(1, activation='sigmoid', name='response')(recent_lstm)
    model = tf.keras.Model(inputs=[full_input, recent_input], outputs=[response_output, time_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss={'response': 'binary_crossentropy', 'reaction_time': 'mse'},
                  metrics={'response': 'accuracy', 'reaction_time': 'mse'})
    return model



In [ ]:
X_full_combined = np.concatenate([X_iid_reshaped, X_rdw_reshaped], axis=0)
X_recent_combined = np.concatenate([X_iid_recent, X_rdw_recent], axis=0)
y_binary_combined = np.concatenate([y_iid_recent, y_rdw_recent], axis=0)
y_time_combined = np.concatenate([rt_iid_recent, rt_rdw_recent], axis=0)
path_type_combined = np.array(['iid'] * len(X_iid_recent) + ['rdw'] * len(X_rdw_recent))

In [ ]:
n_iid = len(X_iid_reshaped)
n_rdw = len(X_rdw_reshaped)

indices_iid = np.random.permutation(n_iid)
indices_rdw = np.random.permutation(n_rdw)

n_folds = min(n_iid, n_rdw) // (batch_size // 2)  

iid_folds = np.array_split(indices_iid, n_folds)
rdw_folds = np.array_split(indices_rdw, n_folds)
all_metrics_by_fold = []
fold_histories = []
for fold in trange(n_folds, desc="LOO Fold Progress"):

    val_idx_iid = iid_folds[fold]
    val_idx_rdw = rdw_folds[fold]


    train_idx_iid = np.setdiff1d(indices_iid, val_idx_iid)
    train_idx_rdw = np.setdiff1d(indices_rdw, val_idx_rdw)

    X_full_train = np.concatenate([X_iid_reshaped[train_idx_iid], X_rdw_reshaped[train_idx_rdw]])
    X_recent_train = np.concatenate([X_iid_recent[train_idx_iid], X_rdw_recent[train_idx_rdw]])
    y_binary_train = np.concatenate([y_iid_recent[train_idx_iid], y_rdw_recent[train_idx_rdw]])
    y_time_train = np.concatenate([rt_iid_recent[train_idx_iid], rt_rdw_recent[train_idx_rdw]])

    X_full_val = np.concatenate([X_iid_reshaped[val_idx_iid], X_rdw_reshaped[val_idx_rdw]])
    X_recent_val = np.concatenate([X_iid_recent[val_idx_iid], X_rdw_recent[val_idx_rdw]])
    y_binary_val = np.concatenate([y_iid_recent[val_idx_iid], y_rdw_recent[val_idx_rdw]])
    y_time_val = np.concatenate([rt_iid_recent[val_idx_iid], rt_rdw_recent[val_idx_rdw]])
    path_type_val = np.array(['iid'] * len(val_idx_iid) + ['rdw'] * len(val_idx_rdw))

    model = create_model()
    history = model.fit(
        {'full_trajectory': X_full_train, 'recent_trajectory': X_recent_train},
        {'response': y_binary_train, 'reaction_time': y_time_train},
        validation_data=(
        {'full_trajectory': X_full_val, 'recent_trajectory': X_recent_val},
        {'response': y_binary_val, 'reaction_time': y_time_val}
    ),
        batch_size=64, epochs=100, verbose=0
    )
    fold_histories.append({
        'Fold': fold,
        'History': history.history
    })
    
    for path_type in ['iid', 'rdw']:
        mask = (path_type_val == path_type)
        if np.any(mask):
            sub_metrics, _ = evaluate_model(
                model,
                X_full_val[mask],
                X_recent_val[mask],
                y_binary_val[mask],
                y_time_val[mask]
            )
            sub_metrics['Fold'] = fold
            sub_metrics['Path Type'] = path_type

            all_metrics_by_fold.append(sub_metrics)


In [ ]:
plot_training_history(record['History'])

In [ ]:
# best model
from tensorflow.keras.models import load_model

model_combined = load_model('DUAL_LSTM_MODEL.keras')
model_combined_rdw = load_model('DUAL_LSTM_MODEL.keras')

In [ ]:
train_results = model_combined_full.evaluate(
    {'full_trajectory': X_full_train_comb, 'recent_trajectory': X_recent_train_comb},  
    {'response': y_binary_train_comb, 'reaction_time': y_time_train_comb}, 
    verbose=1
)

val_results = model_combined_full.evaluate(
    {'full_trajectory': X_full_val_comb, 'recent_trajectory': X_recent_val_comb},  
    {'response': y_binary_val_comb, 'reaction_time': y_time_val_comb},  
    verbose=1
)


print(f'Training Loss: {train_results[0]}')
print(f'Training Response prediction Acc: {train_results[2]}')
print(f'Training RT MSE: {train_results[1]}')

print(f'Val Loss: {val_results[0]}')
print(f'Val Response prediction Acc: {val_results[2]}')
print(f'Val RT MSE: {val_results[1]}')

In [ ]:
metrics_comb, predictions_comb = evaluate_model(
    model_combined_full, 
    X_full_val_comb,  
    X_recent_val_comb,  
    y_binary_val_comb,  
    y_time_val_comb  
)

for key, value in metrics_comb.items():
    
    print(f"{key}: {value}")

In [ ]:
metrics_comb, predictions_comb = evaluate_model(
    model_combined_full, 
    X_full_val_rdw,  
    X_recent_val_rdw,  
    y_binary_val_rdw,  
    y_time_val_rdw  
)

for key, value in metrics_comb.items():
    
    print(f"{key}: {value}")

In [ ]:
metrics_iid, predictions_iid = evaluate_model(
    model_combined_full, 
    X_full_val_iid,  
    X_recent_val_iid,  
    y_binary_val_iid,  
    y_time_val_iid  
)

print('metrics_iid validation')
for key, value in metrics_iid.items():
    
    print(f"{key}: {value}")